# Reranking in Elasticsearch LTR Model

Refer to [ES LTR Docs](https://elasticsearch-learning-to-rank.readthedocs.io/en/latest/searching-with-your-model.html) for more details.

In [1]:
import json
import os
import random
import requests

In [2]:
ES_URL = "http://localhost:9200/"
QUERY_LIST = [
    "murder", "musical", "biography", "police", "world war ii",
    "comedy", "superhero", "nazis", "romance", "martial arts",
    "extramarital", "spy", "vampire", "magic", "wedding",
    "sport", "prison", "teacher", "alien", "dystopia"
]
TOP_N = 10

In [3]:
def rating2label(rating):
    """ convert 0-10 continuous rating to 1-5 categorical labels """
    if rating == 10.0:
        rating -= 0.01
    return int(rating // 2) + 1


def get_rating_string(rating):
    rating_string = []
    for i in range(rating):
        rating_string.append(u"\u2605")
    for i in range(5 - rating):
        rating_string.append(u"\u2606")
    return "".join(rating_string)


print(get_rating_string(3))
print(get_rating_string(rating2label(6.4)))

★★★☆☆
★★★★☆


In [4]:
query = QUERY_LIST[random.randint(0, len(QUERY_LIST))]

## Top 20 results without LTR

In [5]:
def render_results(docs, query, top_n):
    print("top {:d} results for {:s}".format(TOP_N * 2, query))
    print("---")
    for doc in docs:
        doc_id, title, rating, score = doc
        stars = get_rating_string(rating2label(rating))
        print("{:s} {:06d} {:.3f} {:s}".format(stars, int(doc_id), score, title))

In [6]:
headers = {
    "Content-Type": "application/json"
}
data = {
    "query": {
        "dis_max": {
            "queries": [
                { "match": { "title": query }},
                { "match": { "body":  query }}
            ]
        }
    },
    "from": 0,
    "size": TOP_N * 2
}
resp = requests.post(ES_URL + "tmdbindex/_search", headers=headers, data=json.dumps(data))
resp_json = json.loads(resp.text)
result_docs = []
for doc in resp_json["hits"]["hits"]:
    doc_src = doc["_source"]
    doc_id = doc_src["doc_id"]
    rating = doc_src["rating"]
    title = doc_src["title"]
    score = doc["_score"]
    result_docs.append((doc_id, title, rating, score))
render_results(result_docs, query, TOP_N)

top 20 results for alien
---
★★★★☆ 008077 9.157 Alien³
★★★★☆ 000348 9.157 Alien
★★★☆☆ 008078 7.716 Alien: Resurrection
★★★☆☆ 029938 7.716 Alien Escape
★★★☆☆ 010128 7.716 Alien Nation
★★★☆☆ 022076 7.716 Alien Hunter
★★★☆☆ 009773 7.716 Alien Autopsy
★★★☆☆ 013954 7.716 Alien Raiders
★★★☆☆ 016634 7.716 Alien Trespass
★★★☆☆ 052477 7.716 Alien Cargo
★★★☆☆ 052318 7.716 Alien Predators
★★★☆☆ 258193 7.716 Alien Abduction
★★★☆☆ 312526 7.716 Alien Outpost
★☆☆☆☆ 113040 7.716 Alien Origin
★★☆☆☆ 014907 7.716 Alien Apocalypse
★★☆☆☆ 104185 7.716 Alien Tornado
★☆☆☆☆ 113119 7.716 Fugitive Alien
★★★☆☆ 126889 7.716 Alien: Covenant
★★★★☆ 067317 7.716 Dr. Alien
★★☆☆☆ 031397 6.666 Alien from L.A.


## Top 20 results with LTR

In [7]:
data = {
    "query": {
        "dis_max": {
            "queries": [
                { "match": { "title": query }},
                { "match": { "body":  query }}
            ]
        }
    },
    "rescore": {
        "window_size": 100,
        "query": {
            "rescore_query": {
                "sltr": {
                    "params": {
                        "query": query
                    },
                    "model": "es_lambdamart_model",
                }
            }
        }
    },
    "from": 0,
    "size": TOP_N * 2
}
resp = requests.post(ES_URL + "tmdbindex/_search", headers=headers, data=json.dumps(data))
resp_json = json.loads(resp.text)
result_docs = []
for doc in resp_json["hits"]["hits"]:
    doc_src = doc["_source"]
    doc_id = doc_src["doc_id"]
    rating = doc_src["rating"]
    title = doc_src["title"]
    score = doc["_score"]
    result_docs.append((doc_id, title, rating, score))
render_results(result_docs, query, TOP_N)

top 20 results for alien
---
★★★★☆ 000348 6.357 Alien
★★☆☆☆ 014907 6.149 Alien Apocalypse
★★★★☆ 008077 5.758 Alien³
★★★★☆ 067317 4.856 Dr. Alien
★★★☆☆ 312526 4.838 Alien Outpost
★★☆☆☆ 104185 4.838 Alien Tornado
★☆☆☆☆ 113119 4.754 Fugitive Alien
★★★☆☆ 052318 4.749 Alien Predators
★★★☆☆ 009773 4.654 Alien Autopsy
★★★☆☆ 016634 4.557 Alien Trespass
★★☆☆☆ 105538 4.542 Codependent Lesbian Space Alien Seeks Same
★☆☆☆☆ 113040 4.364 Alien Origin
★★★☆☆ 008078 4.317 Alien: Resurrection
★★★☆☆ 010128 4.268 Alien Nation
★★★☆☆ 258193 4.249 Alien Abduction
★★★☆☆ 126889 4.249 Alien: Covenant
★★★☆☆ 022076 4.228 Alien Hunter
★★★☆☆ 013954 4.195 Alien Raiders
★★★☆☆ 052477 4.034 Alien Cargo
★★★☆☆ 029938 4.018 Alien Escape
